# Data Science Capstone Project
### Title: Predicting Severity of Collisions

In [116]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [117]:
import pandas as pd
import numpy as np
import itertools
import types
from botocore.client import Config
import ibm_boto3
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
%matplotlib inline

In [118]:
# The code was removed by Watson Studio for sharing.

In [119]:
cdf = pd.read_csv(body)

pd.set_option('display.max_columns', None)
cdf.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [120]:
cdf.shape

(194673, 38)

In [121]:
cdf["ROADCOND"].isnull().sum()

5012

In [122]:
cdf.groupby('SEVERITYCODE').size()

SEVERITYCODE
1    136485
2     58188
dtype: int64

In [123]:
cdf.columns.values

array(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY',
       'REPORTNO', 'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION',
       'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC',
       'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT',
       'VEHCOUNT', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE',
       'SDOT_COLDESC', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER',
       'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING',
       'ST_COLCODE', 'ST_COLDESC', 'SEGLANEKEY', 'CROSSWALKKEY',
       'HITPARKEDCAR'], dtype=object)

In [124]:
cdf['INCDATE'] = pd.to_datetime(cdf['INCDATE'])
cdf['INCDTTM'] = pd.to_datetime(cdf['INCDTTM'])

In [184]:
cdf2 = cdf[['SEVERITYCODE', 'STATUS', 'ADDRTYPE', 'SEVERITYCODE.1', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', \
            'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE', 'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'INATTENTIONIND', \
            'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND', 'PEDROWNOTGRNT', 'SPEEDING', 'ST_COLCODE', 'HITPARKEDCAR']]

In [185]:
cdf2.head()

,SEVERITYCODE,STATUS,ADDRTYPE,SEVERITYCODE.1,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR
0,2,Matched,Intersection,2,Angles,2,0,0,2,2013-03-27 00:00:00+00:00,2013-03-27 14:54:00,At Intersection (intersection related),11,NaN,N,Overcast,Wet,Daylight,NaN,NaN,10,N
1,1,Matched,Block,1,Sideswipe,2,0,0,2,2006-12-20 00:00:00+00:00,2006-12-20 18:55:00,Mid-Block (not related to intersection),16,NaN,0,Raining,Wet,Dark - Street Lights On,NaN,NaN,11,N
2,1,Matched,Block,1,Parked Car,4,0,0,3,2004-11-18 00:00:00+00:00,2004-11-18 10:20:00,Mid-Block (not related to intersection),14,NaN,0,Overcast,Dry,Daylight,NaN,NaN,32,N
3,1,Matched,Block,1,Other,3,0,0,3,2013-03-29 00:00:00+00:00,2013-03-29 09:26:00,Mid-Block (not related to intersection),11,NaN,N,Clear,Dry,Daylight,NaN,NaN,23,N
4,2,Matched,Intersection,2,Angles,2,0,0,2,2004-01-28 00:00:00+00:00,2004-01-28 08:04:00,At Intersection (intersection related),11,NaN,0,Raining,Wet,Daylight,NaN,NaN,10,N


In [186]:
cdf2.groupby('STATUS').size()

STATUS
Matched      189786
Unmatched      4887
dtype: int64

In [187]:
cdf2 = cdf2[cdf2.STATUS != "Unmatched"]
cdf2 = cdf2.drop('STATUS', 1)

In [188]:
cdf2.groupby('ADDRTYPE').size()

ADDRTYPE
Alley              747
Block           123663
Intersection     63559
dtype: int64

In [189]:
cdf2.groupby('SEVERITYCODE.1').size()

SEVERITYCODE.1
1    132627
2     57159
dtype: int64

In [190]:
cdf2.groupby('SEVERITYCODE').size()

SEVERITYCODE
1    132627
2     57159
dtype: int64

In [191]:
cdf2 = cdf2.drop('SEVERITYCODE.1', 1)

In [192]:
cdf2.groupby('COLLISIONTYPE').size()

COLLISIONTYPE
Angles        34674
Cycles         5415
Head On        2024
Left Turn     13703
Other         23703
Parked Car    47986
Pedestrian     6607
Rear Ended    34089
Right Turn     2956
Sideswipe     18608
dtype: int64

In [193]:
cdf2.groupby('JUNCTIONTYPE').size()

JUNCTIONTYPE
At Intersection (but not related to intersection)     2060
At Intersection (intersection related)               61334
Driveway Junction                                    10538
Mid-Block (but intersection related)                 22388
Mid-Block (not related to intersection)              87119
Ramp Junction                                          162
Unknown                                                  7
dtype: int64

In [194]:
cdf2.groupby('INATTENTIONIND').size()

INATTENTIONIND
Y    29805
dtype: int64

In [195]:
cdf2['INATTENTIONIND'] = cdf2['INATTENTIONIND'].replace(np.nan, 0)
cdf2['INATTENTIONIND'] = cdf2['INATTENTIONIND'].replace("Y", 1)
cdf2.groupby('INATTENTIONIND').size()

INATTENTIONIND
0    159981
1     29805
dtype: int64

In [196]:
cdf2.groupby('UNDERINFL').size()

UNDERINFL
0     80391
1      3995
N    100274
Y      5126
dtype: int64

In [197]:
cdf2['UNDERINFL'] = cdf2['UNDERINFL'].replace(['0','N'],0)
cdf2['UNDERINFL'] = cdf2['UNDERINFL'].replace(['1','Y'],1)
cdf2.groupby('UNDERINFL').size()

UNDERINFL
0    180665
1      9121
dtype: int64

In [198]:
cdf2.groupby('PEDROWNOTGRNT').size()

PEDROWNOTGRNT
Y    4667
dtype: int64

In [199]:
cdf2['PEDROWNOTGRNT'] = cdf2['PEDROWNOTGRNT'].replace(np.nan, 0)
cdf2['PEDROWNOTGRNT'] = cdf2['PEDROWNOTGRNT'].replace("Y", 1)
cdf2.groupby('PEDROWNOTGRNT').size()

PEDROWNOTGRNT
0    185119
1      4667
dtype: int64

In [200]:
cdf2.groupby('SPEEDING').size()

SPEEDING
Y    9333
dtype: int64

In [201]:
cdf2['SPEEDING'] = cdf2['SPEEDING'].replace(np.nan, 0)
cdf2['SPEEDING'] = cdf2['SPEEDING'].replace("Y", 1)
cdf2.groupby('SPEEDING').size()

SPEEDING
0    180453
1      9333
dtype: int64

In [202]:
cdf2.groupby('HITPARKEDCAR').size()

HITPARKEDCAR
N    182868
Y      6918
dtype: int64

In [203]:
cdf2['HITPARKEDCAR'] = cdf2['HITPARKEDCAR'].replace("N", 0)
cdf2['HITPARKEDCAR'] = cdf2['HITPARKEDCAR'].replace("Y", 1)
cdf2.groupby('HITPARKEDCAR').size()

HITPARKEDCAR
0    182868
1      6918
dtype: int64

In [204]:
cdf2.describe()

,SEVERITYCODE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,PEDROWNOTGRNT,SPEEDING,HITPARKEDCAR
count,189786.000000,189786.000000,189786.000000,189786.000000,189786.000000,189786.000000,189786.000000,189786.000000,189786.000000,189786.000000,189786.000000
mean,1.301176,2.459017,0.038096,0.029122,1.970209,13.926854,0.157045,0.048059,0.024591,0.049176,0.036452
std,0.458770,1.354894,0.200594,0.169492,0.557751,6.857187,0.363844,0.213892,0.154875,0.216237,0.187411
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,0.000000,0.000000,2.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,2.000000,0.000000,0.000000,2.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,3.000000,0.000000,0.000000,2.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,81.000000,6.000000,2.000000,12.000000,69.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [205]:
 cdf2.isnull().sum()

SEVERITYCODE         0
ADDRTYPE          1817
COLLISIONTYPE       21
PERSONCOUNT          0
PEDCOUNT             0
PEDCYLCOUNT          0
VEHCOUNT             0
INCDATE              0
INCDTTM              0
JUNCTIONTYPE      6178
SDOT_COLCODE         0
INATTENTIONIND       0
UNDERINFL            0
WEATHER            197
ROADCOND           128
LIGHTCOND          286
PEDROWNOTGRNT        0
SPEEDING             0
ST_COLCODE          18
HITPARKEDCAR         0
dtype: int64

In [206]:
cdf2.head()

,SEVERITYCODE,ADDRTYPE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SPEEDING,ST_COLCODE,HITPARKEDCAR
0,2,Intersection,Angles,2,0,0,2,2013-03-27 00:00:00+00:00,2013-03-27 14:54:00,At Intersection (intersection related),11,0,0,Overcast,Wet,Daylight,0,0,10,0
1,1,Block,Sideswipe,2,0,0,2,2006-12-20 00:00:00+00:00,2006-12-20 18:55:00,Mid-Block (not related to intersection),16,0,0,Raining,Wet,Dark - Street Lights On,0,0,11,0
2,1,Block,Parked Car,4,0,0,3,2004-11-18 00:00:00+00:00,2004-11-18 10:20:00,Mid-Block (not related to intersection),14,0,0,Overcast,Dry,Daylight,0,0,32,0
3,1,Block,Other,3,0,0,3,2013-03-29 00:00:00+00:00,2013-03-29 09:26:00,Mid-Block (not related to intersection),11,0,0,Clear,Dry,Daylight,0,0,23,0
4,2,Intersection,Angles,2,0,0,2,2004-01-28 00:00:00+00:00,2004-01-28 08:04:00,At Intersection (intersection related),11,0,0,Raining,Wet,Daylight,0,0,10,0


In [207]:
cdf2['dayofweek'] = cdf2['INCDATE'].dt.dayofweek
cdf2['weekend']= cdf2['dayofweek'].apply(lambda x: 1 if (x>3)  else 0)

cdf2 = cdf2.drop('dayofweek', 1)

In [210]:
cdf2['monthofyear'] = cdf2['INCDATE'].dt.month
cdf2['seasonofyear'] = (cdf2['monthofyear']%12 + 3)//3

cdf2.groupby('seasonofyear').size()


seasonofyear
1    44665
2    47643
3    48285
4    49193
dtype: int64

In [ ]:
cdf2 = cdf2.drop('INCDATE', 1)